<a href="https://colab.research.google.com/github/lukezhq10/fcc-neural-network-SMS-text-classifier/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

# Prepare data

In [ ]:
train_df = pd.read_csv(train_file_path, sep='\t', header=None, names=['type', 'msg'])
train_df.head()

In [ ]:
train_df.shape

In [ ]:
train_df.isnull().sum()

In [ ]:
test_df = pd.read_csv(test_file_path, sep='\t', header=None, names=['type', 'msg'])
test_df.head()

In [ ]:
test_df.shape

In [ ]:
test_df.isnull().sum()

### Convert ham/spam data into numerical

In [ ]:
train_df["type"] = pd.factorize(train_df["type"])[0]
test_df["type"] = pd.factorize(test_df["type"])[0]

In [ ]:
train_df.head()

## Prepare labels for training

In [ ]:
train_labels = train_df['type'].values
train_dataset = tf.data.Dataset.from_tensor_slices(
    (train_df['msg'].values, train_labels)
)

test_labels = test_df['type'].values
test_dataset = tf.data.Dataset.from_tensor_slices(
    (test_df['msg'].values, test_labels)
)
test_dataset.element_spec

## Shuffling the data for training

In [ ]:
BUFFER_SIZE= 100
BATCH_SIZE = 32
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Creating text encorder (TextVectorization layer)

In [ ]:
encoder = tf.keras.layers.TextVectorization(
    output_mode='int',
    max_tokens=1000,
    output_sequence_length=1000)

encoder.adapt(train_dataset.map(lambda text, label: text))

In [ ]:
# First 20 tokens (vocab) the vectorizer has learned in order of frequency
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

# Creating the model

In [ ]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1)
])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])
model.summary()

# Training the model

In [ ]:
history = model.fit(train_dataset,
                    validation_data=test_dataset,
                    validation_steps=30,
                    epochs=10)

# Plotting the accuracy and loss metrics

In [ ]:
def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])

In [ ]:
plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plot_graphs(history, 'accuracy')
plt.ylim(None, 1)
plt.subplot(1, 2, 2)
plot_graphs(history, 'loss')
plt.ylim(0, None)

In [ ]:
# testing with a sample_text (expecting ham // prediction < 0.5)
sample_text = ('i am waiting machan. call me once you free.')
prediction = model.predict(np.array([sample_text]))
prediction

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  ps = model.predict([pred_text])
  print(ps)
  p = ps[0][0]
  return [p, 'ham' if p < 0.5 else 'spam']

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
